# Feature Extraction

1. HeatMap
* Insights from the heat map shows the 100 % correlation between solar-radiation and uv-index and hence removing the uv-index column
* 99% correlation between temperature and feelslike and so removing the feelslike column
* Dew Point has 0.36 correleation with Demand (MW) and hence add this column

Conclusion : (temperature, Dew Point, solar-radiation)

2. Scatter Plot
* There doesn't seem to be non-linear relationship between wind-direction and Demand (MW)

* Add all the other continuous variables

3. Bar Graph - Categorical Mean Demand Bar Chart
* Since, all categories have significant differences in mean value (differences in mean > 6MW), put all the categories as features



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [24]:
demand_forecast_data_2_path = "Data/Demand Forecasting/Demand Forecasting Weather Data upto Feb 28.csv"
demand_forecast_data_2 = pd.read_csv(project_path + demand_forecast_data_2_path)
demand_forecast_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27720 entries, 0 to 27719
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              27720 non-null  object 
 1   datetime          27720 non-null  object 
 2   Temperature       27720 non-null  float64
 3   feelslike         27720 non-null  float64
 4   dewpoint          27720 non-null  float64
 5   humidity          27720 non-null  float64
 6   precipitation     27720 non-null  float64
 7   precipprob        27720 non-null  int64  
 8   preciptype        2751 non-null   object 
 9   snow              27702 non-null  float64
 10  snowdepth         27702 non-null  float64
 11  windgust          12223 non-null  float64
 12  windspeed         27720 non-null  float64
 13  winddirection     27720 non-null  float64
 14  sealevelpressure  27702 non-null  float64
 15  cloudcover        27720 non-null  float64
 16  visibility        27702 non-null  float6

In [25]:
demand_forecast_data_2.datetime.nunique()

27717

In [27]:
data_path = "/content/drive/MyDrive/Colab Notebooks/Hitachi/Data/imputed_data.csv"
data_df = pd.read_csv(data_path)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27555 entries, 0 to 27554
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Prices
(EUR/MWh)                27555 non-null  float64
 1   datetime                        27555 non-null  object 
 2   Demand (MW)                     27555 non-null  float64
 3   Temperature                     27555 non-null  float64
 4   feelslike                       27555 non-null  float64
 5   dewpoint                        27555 non-null  float64
 6   humidity                        27555 non-null  float64
 7   precipitation                   27555 non-null  float64
 8   precipprob                      27555 non-null  float64
 9   snow                            27555 non-null  float64
 10  snowdepth                       27555 non-null  float64
 11  windgust                        27555 non-null  float64
 12  windspeed                       

In [28]:
data_df.head()

,Prices\n(EUR/MWh),datetime,Demand (MW),Temperature,feelslike,dewpoint,humidity,precipitation,precipprob,snow,...,ice,rain,snow.1,Clear,Freezing Drizzle/Freezing Rain,Ice,Overcast,Partially cloudy,Rain,Snow
0,34.00,2020-01-01 00:00:00,445.8,38.0,34.4,29.0,69.76,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
1,26.38,2020-01-01 01:00:00,424.5,38.0,34.4,29.9,72.37,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
2,26.50,2020-01-01 02:00:00,423.5,38.0,35.7,30.8,75.06,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3,27.01,2020-01-01 03:00:00,418.8,37.1,33.4,29.9,74.97,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,27.40,2020-01-01 04:00:00,414.8,35.1,32.3,29.0,78.41,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0


In [29]:
datetime_series = data_df.datetime
duplicates = datetime_series[datetime_series.duplicated(keep=False)]

# Print duplicates and their counts
if not duplicates.empty:
    print("Duplicates found:")
    print(duplicates.value_counts())
else:
    print("No duplicates found.")

Duplicates found:
2020-11-01 01:00:00    2
2021-11-07 01:00:00    2
2022-11-06 01:00:00    2
Name: datetime, dtype: int64


In [30]:
data_df = data_df.drop_duplicates(subset='datetime', keep='first')
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27552 entries, 0 to 27554
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Prices
(EUR/MWh)                27552 non-null  float64
 1   datetime                        27552 non-null  object 
 2   Demand (MW)                     27552 non-null  float64
 3   Temperature                     27552 non-null  float64
 4   feelslike                       27552 non-null  float64
 5   dewpoint                        27552 non-null  float64
 6   humidity                        27552 non-null  float64
 7   precipitation                   27552 non-null  float64
 8   precipprob                      27552 non-null  float64
 9   snow                            27552 non-null  float64
 10  snowdepth                       27552 non-null  float64
 11  windgust                        27552 non-null  float64
 12  windspeed                       

In [31]:
columns_to_drop = ["feelslike", "uvindex", "winddirection"]
features_data_df = data_df.drop(columns=columns_to_drop)

In [32]:
features_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27552 entries, 0 to 27554
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Prices
(EUR/MWh)                27552 non-null  float64
 1   datetime                        27552 non-null  object 
 2   Demand (MW)                     27552 non-null  float64
 3   Temperature                     27552 non-null  float64
 4   dewpoint                        27552 non-null  float64
 5   humidity                        27552 non-null  float64
 6   precipitation                   27552 non-null  float64
 7   precipprob                      27552 non-null  float64
 8   snow                            27552 non-null  float64
 9   snowdepth                       27552 non-null  float64
 10  windgust                        27552 non-null  float64
 11  windspeed                       27552 non-null  float64
 12  sealevelpressure                

In [34]:
features_data_df.drop(columns=["Prices\n(EUR/MWh)"], inplace=True)

In [35]:
features_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27552 entries, 0 to 27554
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   datetime                        27552 non-null  object 
 1   Demand (MW)                     27552 non-null  float64
 2   Temperature                     27552 non-null  float64
 3   dewpoint                        27552 non-null  float64
 4   humidity                        27552 non-null  float64
 5   precipitation                   27552 non-null  float64
 6   precipprob                      27552 non-null  float64
 7   snow                            27552 non-null  float64
 8   snowdepth                       27552 non-null  float64
 9   windgust                        27552 non-null  float64
 10  windspeed                       27552 non-null  float64
 11  sealevelpressure                27552 non-null  float64
 12  cloudcover                      

In [38]:
features_data_df.tail(25)

,datetime,Demand (MW),Temperature,dewpoint,humidity,precipitation,precipprob,snow,snowdepth,windgust,...,ice,rain,snow.1,Clear,Freezing Drizzle/Freezing Rain,Ice,Overcast,Partially cloudy,Rain,Snow
27530,2023-02-20 23:00:00,449.8,43.1,37.1,79.40,0.0,0.0,0.0,0.0,4.7,...,0,0,0,1,0,0,0,0,0,0
27531,2023-02-21 00:00:00,439.2,42.2,37.1,82.19,0.0,0.0,0.0,0.0,6.9,...,0,0,0,1,0,0,0,0,0,0
27532,2023-02-21 01:00:00,427.4,42.2,37.1,82.19,0.0,0.0,0.0,0.0,6.9,...,0,0,0,1,0,0,0,0,0,0
27533,2023-02-21 02:00:00,427.6,39.8,35.1,82.93,0.0,0.0,0.0,0.0,6.9,...,0,0,0,1,0,0,0,0,0,0
27534,2023-02-21 03:00:00,432.9,41.0,37.1,86.01,0.0,0.0,0.0,0.0,8.1,...,0,0,0,1,0,0,0,0,0,0
27535,2023-02-21 04:00:00,439.6,36.0,33.2,89.45,0.0,0.0,0.0,0.0,10.3,...,0,0,0,1,0,0,0,0,0,0
27536,2023-02-21 05:00:00,450.4,44.1,32.0,62.28,0.0,0.0,0.0,0.0,13.9,...,0,0,0,1,0,0,0,0,0,0
27537,2023-02-21 06:00:00,473.0,47.0,30.8,53.12,0.0,0.0,0.0,0.0,15.0,...,0,0,0,1,0,0,0,0,0,0
27538,2023-02-21 07:00:00,519.4,45.0,30.8,57.42,0.0,0.0,0.0,0.0,16.1,...,0,0,0,1,0,0,0,0,0,0
27539,2023-02-21 08:00:00,516.2,47.0,32.0,55.67,0.0,0.0,0.0,0.0,17.2,...,0,0,0,1,0,0,0,0,0,0


In [39]:
features_data_df['datetime'] = pd.to_datetime(features_data_df['datetime'])  # Convert the column to datetime if it's not already
# Sort the DataFrame based on the datetime column
sorted_features_data_df = features_data_df.sort_values(by='datetime')
sorted_features_data_df = sorted_features_data_df.reset_index(drop=True)

In [40]:
sorted_features_data_df.head(25)

,datetime,Demand (MW),Temperature,dewpoint,humidity,precipitation,precipprob,snow,snowdepth,windgust,...,ice,rain,snow.1,Clear,Freezing Drizzle/Freezing Rain,Ice,Overcast,Partially cloudy,Rain,Snow
0,2020-01-01 00:00:00,445.8,38.0,29.0,69.76,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,0,0,0,1,0,0,0
1,2020-01-01 01:00:00,424.5,38.0,29.9,72.37,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,0,0,0,1,0,0,0
2,2020-01-01 02:00:00,423.5,38.0,30.8,75.06,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,0,0,0,1,0,0,0
3,2020-01-01 03:00:00,418.8,37.1,29.9,74.97,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,0,0,0,0,1,0,0
4,2020-01-01 04:00:00,414.8,35.1,29.0,78.41,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,1,0,0,0,0,0,0
5,2020-01-01 05:00:00,432.1,34.1,28.1,78.64,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,1,0,0,0,0,0,0
6,2020-01-01 06:00:00,436.8,32.0,26.1,78.51,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,1,0,0,0,0,0,0
7,2020-01-01 07:00:00,446.8,32.0,25.1,75.33,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,1,0,0,0,0,0,0
8,2020-01-01 08:00:00,453.5,33.2,26.1,74.88,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,1,0,0,0,0,0,0
9,2020-01-01 09:00:00,447.8,37.1,27.0,66.41,0.0,0.0,0.0,0.0,15.410436,...,0,0,0,1,0,0,0,0,0,0


In [41]:
sorted_features_data_df.isnull().sum()

datetime                          0
Demand (MW)                       0
Temperature                       0
dewpoint                          0
humidity                          0
precipitation                     0
precipprob                        0
snow                              0
snowdepth                         0
windgust                          0
windspeed                         0
sealevelpressure                  0
cloudcover                        0
visibility                        0
solarradiation                    0
severerisk                        0
freezingrain                      0
ice                               0
rain                              0
snow.1                            0
Clear                             0
Freezing Drizzle/Freezing Rain    0
Ice                               0
Overcast                          0
Partially cloudy                  0
Rain                              0
Snow                              0
dtype: int64

In [42]:
data_path = "/content/drive/MyDrive/Colab Notebooks/Hitachi/Data/_data.csv"
sorted_features_data_df.to_csv(data_path, index=True)